# NN_tuning

In [2]:
#实现时间偏移
!pip install  pendulum

In [3]:
#sklearn 加速
#!pip install scikit-learn scikit-learn-intelex -i https://pypi.douban.com/simple/
from sklearnex import patch_sklearn, unpatch_sklearn
patch_sklearn()


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [4]:
import os
import numpy as np
from tqdm import tqdm
import torch
import torch.jit
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import tensorflow as tf

import pendulum

#from openfe import openfe, transform
#import graphviz

from datetime import timedelta
import warnings; 
warnings.filterwarnings("ignore")

import pandas as pd 
from pandas.tseries.offsets import Day, MonthEnd, MonthBegin # 时间处理
import numpy as np
import pyarrow.feather as feather
import matplotlib.pyplot as plt 

import optuna
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, KBinsDiscretizer, LabelEncoder, OrdinalEncoder


RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

## GPU

In [5]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc. 
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


# Neural network

In [6]:
class AssetPricingNN(nn.Module):
    def __init__(self, input_size, hidden_size_0,hidden_size_1,hidden_size_2,output_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size_0)
        self.fc2 = nn.Linear(hidden_size_0, hidden_size_1)
        self.fc3 = nn.Linear(hidden_size_1, hidden_size_2)
        self.predict = nn.Linear(hidden_size_2,output_size)
        self.emb =  nn.Embedding(3,2)

    def forward(self,x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.predict(x)
        return x

## Model

In [7]:
def split_cat(data):
    s = (data.dtypes == 'int64')
    object_cols = list(s[s].index)# 移除含有类别变量的列
    data_category = pd.DataFrame(data,columns=['permno','sic']).copy()
    
    # 移除数据集含有类别变量的列（因为我们已经获得了含有独热编码的列）
    data_non_category = data.drop(object_cols, axis=1)
    
    return data_category,data_non_category

In [8]:
class Factor_models(object):
    '''
    params:
    -------
        chardf:个股特征值的面板数据，index为日期格式，columns为features。
        retdf:个股收益率的面板数据，index为日期格式。
        first_train_end_date:初次训练时，训练集的截止日期。（开始日期默认为chardf和retdf的第一期）。格式为'%Y%m%d'。
        last_train_end_date:最后一次训练时，训练集的截止日期。格式为'%Y%m%d'。
        freq:按什么频率expanding训练集，默认为'm'，每个月滚动一次。
    
    methods:
    --------
    
    '''
    def __init__(self,data,train_period,freq='m'):
        '''
        参数初始化
        '''
    
        self.data = data
        self.train_period = train_period
        self.freq = freq # 训练集expanding的频率，是按月还是按年还是其他
        
    def predict_ret(self):
        dates = self.data.index.unique()
        dates = dates.sort_values()
        print("=====dates=======")
        print(dates)
        test_dates= dates[self.train_period:len(dates)]
        print("====test months=====")
        print(test_dates)
        
        # 创建一个train_end_list，训练集每月expanding。
        preddf = pd.DataFrame() # 存储不同模型预测出来的y值，即存储样本外预测收益率的值
        
        
        index = 0  # 计数用

        for end_date in tqdm(test_dates,desc='Spilt and Train'): # 通过逐月改变训练集end_date的方法，切割样本
            
            """
            用两个月做validation set；
            用
            """
            end_del2 =  end_date - pd.Timedelta(days=31)
            temp = self.data[(self.data.index <  end_del2)]
            train_temp = self.data[self.data.index <  end_date]
            valid_temp = self.data[ (self.data.index >= end_del2) & (self.data.index < end_date) ]
            test_temp = self.data[self.data.index == end_date]
            
            # 训练集
            #train_y = temp.ret
            #train_x = temp.drop(['ret'], axis=1)
            y_train = temp.ret_fut
            X_train = temp.drop(['ret_fut','ret','year_month'], axis=1)
    

            # 验证集
            y_valid = valid_temp.ret_fut
            X_valid = valid_temp.drop(['ret_fut','ret','year_month'], axis=1)
            #valid_y = valid_temp.ret_fut
            #valid_x  = valid_temp.drop(['ret_fut'], axis=1)
        
            #测试集
            y_test = test_temp.ret_fut
            X_test = test_temp.drop(['ret_fut','ret','year_month'], axis=1)

            #预测训练数据集
            y_ptrain = train_temp.ret_fut
            X_ptrain = train_temp.drop(['ret_fut','ret','year_month'], axis=1)

            #分割给optuna用的数据集
            #X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.2, random_state=0)
      

            
            # 数据标准化
            #st = StandardScaler()
            #sv = StandardScaler()
            #s1 = StandardScaler()
            #s2 = StandardScaler()
            
            #train_x = st.fit_transform(train_x)
            #valid_x = sv.fit_transform(valid_x)
            #X_train = s1.fit_transform(X_train)
            #X_test = s2.fit_transform(X_test)
            
            #对数据进行标准化
            s = (X_train.dtypes == 'float64')
            object_cols = list(s[s].index)

            print(object_cols)

            scaler = StandardScaler()
            for col in object_cols:
                X_train[col] = scaler.fit_transform(X_train[col].values.reshape(-1,1))
                X_valid[col] = scaler.fit_transform(X_valid[col].values.reshape(-1,1))
                X_test[col] = scaler.transform(X_test[col].values.reshape(-1,1))
                X_ptrain[col] = scaler.transform(X_ptrain[col].values.reshape(-1,1))
    
            y_train = y_train.values.reshape(-1,1)
            y_valid = y_valid.values.reshape(-1,1)
            y_test = y_test.values.reshape(-1,1)
            y_ptrain = y_ptrain.values.reshape(-1,1)

            #恢复原格式
            #valid_x = pd.DataFrame(valid_x)
            #X_train = pd.DataFrame()

            # 数据标准化
            #train_y = train_y.values.reshape(-1, 1)
            #train_x = train_x.values
            #valid_x = valid_x.values

            #train_y = train_y.values.reshape(-1, 1)
            #valid_y = valid_y.values.reshape(-1,1)
            #y_train = y_train.values.reshape(-1,1)
            #y_test = y_test.values.reshape(-1,1)
            
            # 建模预测收益率
            ## 先创建一个临时的temp_preddf,用来存储当前月份的验证集下的real y和不同模型的预测y
            temp_preddf = pd.DataFrame() # 创建当前训练集下训练出的predict y和real y
            temp_preddf['real_y'] = y_test[:,0]# real_y就是验证集valid_y的第一列。因为valid_y是真实收益率数据在vailid_date上的切割
            
            #分割给optuna用的数据集
            #X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.2, random_state=0)

            ## NN3,未调参
            ### 三层的神经网络

            #train_x = torch.tensor(train_x,dtype = torch.float)
            #train_y = torch.tensor(train_y,dtype = torch.float)
            #valid_x = torch.tensor(valid_x,dtype = torch.float)
            #valid_y = torch.tensor(valid_y,dtype = torch.float)
            
            #转为tensor，以使用gpu训练


            X_train = torch.tensor(X_train.values,dtype = torch.float)
            X_valid = torch.tensor(X_valid.values,dtype = torch.float)
            X_test = torch.tensor(X_test.values,dtype = torch.float)
            X_ptrain = torch.tensor(X_ptrain.values,dtype = torch.float)
            y_train = torch.tensor(y_train,dtype = torch.float)
            y_valid = torch.tensor(y_valid,dtype = torch.float)
            y_test = torch.tensor(y_test,dtype = torch.float)
            y_ptrain = torch.tensor(y_ptrain,dtype = torch.float)

            #train_x =  train_x.to(device)
            #train_y = train_y.to(device)

            #valid_x = valid_x.to(device)
            #valid_y = valid_y.to(device) 

            X_train = X_train.to(device)
            X_valid = X_valid.to(device)
            X_test = X_test.to(device)
            X_ptrain = X_ptrain.to(device)
            y_train = y_train.to(device)
            y_valid = y_valid.to(device)
            y_test = y_test.to(device) 
            y_ptrain = y_ptrain.to(device)

            

            if index == 0 or index % 12 == 11:

                # 定义optuna使用的目标
                def objective(trial):
                    # Set hyperparameter
                    input_size = trial.suggest_int("input_size",58,58)
                    hidden_size_0 = trial.suggest_int("hidden_size_0", 1, 50)
                    hidden_size_1 = trial.suggest_int("hidden_size_1", 1, 50)
                    hidden_size_2 = trial.suggest_int("hidden_size_2", 1, 50)
                    output_size = trial.suggest_int("output_size",1,1)


                    # Create model
                    model = AssetPricingNN(input_size, hidden_size_0,hidden_size_1,hidden_size_2,output_size).to(device)

                    # Train model
                    optimizer = optim.Adam(model.parameters(),lr=0.005,weight_decay=0.001)
                    criterion = nn.MSELoss()


                    for epoch in range(1000):
                        # Forward pass
                        predictions = model(X_train)
                        loss = criterion(predictions, y_train)


                        # Backward pass
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()
                    
                    # Evaluate on validation set
                    with torch.no_grad():
                        predictions = model(X_valid)
                        val_loss = criterion(predictions, y_valid)

                    return val_loss

                # Create study
                study = optuna.create_study()

                # Run study
                study.optimize(objective, n_trials=50)

                # Print best hyperparameters
                best_params = study.best_params
                print(best_params)

                # Create model
                #model = AssetPricingNN(**best_params).to(device)

            
            # Create model
            net = AssetPricingNN(**best_params).to(device) # 不是这里的问题

            # Train model
            optimizer2 = optim.Adam(net.parameters(),lr=0.005,weight_decay=0.001)
            criterion2 = nn.MSELoss()

            for epoch in range(1000):
                # Forward pass
                predictions = net(X_ptrain)
                loss = criterion2(predictions, y_ptrain)

                # Backward pass
                optimizer2.zero_grad()
                loss.backward()
                optimizer2.step()

            with torch.no_grad():
                y_predict = net(X_test)
                print("======y_predict=======",y_predict)
                print("======y_test=======",y_test)
                loss = criterion2(y_predict,y_test)
                print("======loss=======",loss)
                y_predict = y_predict.detach().cpu().numpy()
                temp_preddf['NN3_y'] = y_predict
            
            ## 将temp_preddf并入preddf
            preddf = preddf.append(temp_preddf) # 将当前valid_date下得到的predict_y和real_y一起并入preddf中
            self._preddf = preddf

        torch.save(net, '/home/mw/project/NN/NN3_tuning.pt')
        return preddf # 最后我们只返回preddf，也就是所有期的predict y和real y
    
    def cal_oos(self):
        # 计算out-of-sample R2 根据代码开头的公式
        try:
            preddf = self._preddf # 如果self已经有self._preddf，即self.predict_ret()已经运行过了，已经预测过收益率了，则无需再次运行。
        except:
            preddf = self.predict_ret() # 如果之前没有运行过self.predict_ret()，则需要运行。
        denominator = (preddf['real_y'] ** 2).sum() # 分母是真实收益率的平方和
        numerator = preddf.apply(lambda x: preddf['real_y'] - x).iloc[:,1:] # 分子是real_y - predict_y的平方和
        numerator = (numerator ** 2).sum()
        
        roos = 1 - numerator / denominator # 再用 1 减去分子/分母
        roos.index = roos.index.str.rstrip('_y') # 之前的index都是模型_y，比如"OLS_y"，不美观，删除_y。
        fig,ax = plt.subplots(figsize = (16,12)) # 画图，将不同模型的Roos画出来。
        plt.title('Out-of-sample predicting R2', fontsize = 20)
        ax.bar(x = roos.index, height = roos)
        plt.show()
        return roos # 返回样本外Roos，这个Roos是不同模型对应的样本外R2

## 主体

In [9]:
#导入数据
with open('/home/mw/input/stock3636/chars60_rank_imputed.feather', 'rb') as f:
    data = feather.read_feather(f)
data['date'] = data['date'].astype('datetime64')


# 滞后代码
data['year_month'] = pd.to_datetime(data['date']).dt.to_period('M')
data['ret_fut'] = data.groupby('permno')['ret'].shift(-1)
data = data.dropna(axis=0,subset=['ret_fut']) #删除有空缺值的行

data.set_index('date', inplace=True)

# 缺失值处理
## 查看缺失值--没有缺失值
print('Missing data: {} items\n'.format(len(data[data.isna().any(1)])), data[data.isna().any(1)]) # 看一下缺失值是哪些行

Missing data: 0 items
 Empty DataFrame
Columns: [gvkey, permno, sic, ret, exchcd, shrcd, ffi49, lag_me, rank_acc, rank_cash, rank_maxret, rank_lgr, rank_roe, rank_sgr, rank_bm, rank_noa, rank_rdm, rank_chtx, rank_chpm, rank_cashdebt, rank_mom12m, rank_turn, rank_rvar_capm, rank_ato, rank_pscore, rank_chcsho, rank_beta, rank_dolvol, rank_alm, rank_std_dolvol, rank_rvar_mean, rank_rd_sale, rank_abr, rank_sp, rank_mom60m, rank_pctacc, rank_ill, rank_lev, rank_rna, rank_mom6m, rank_seas1a, rank_nincr, rank_mom1m, rank_zerotrade, rank_cfp, rank_pm, rank_me, rank_ni, rank_mom36m, rank_rvar_ff3, rank_gma, rank_roa, rank_rsup, rank_baspread, rank_sue, rank_grltnoa, rank_std_turn, rank_depr, rank_cinvest, rank_op, rank_agr, rank_ep, log_me, year_month, ret_fut]
Index: []

[0 rows x 65 columns]


In [10]:
data.dtypes

gvkey             int64
permno            int64
sic               int64
ret             float64
exchcd          float64
                ...    
rank_agr        float64
rank_ep         float64
log_me          float64
year_month    period[M]
ret_fut         float64
Length: 65, dtype: object

In [11]:
"""
删除分类变量--embedding后约等于没有作用且速度慢
"""
s = (data.dtypes == 'int64')
object_cols = list(s[s].index)# 移除含有类别变量的列
# 移除数据集含有类别变量的列
data = data.drop(object_cols, axis=1)

In [12]:
basic_2_factors = Factor_models(data,20,freq='m') #2013年只有八年的数据

In [2]:
# 计算样本外R2，运行耗时较长
roos = basic_2_factors.cal_oos() # 计算不同模型样本外R2。self.cal_oos()中已经包含了self.predict_ret()的操作，先通过不同的模型预测收益率，再比较样本外真实收益率和预测收益率的差异
roos.to_csv('/home/mw/project/nn3_tuning_roos.csv')

In [14]:
# 查看不同模型在不同月份的样本外预测ret
predict = basic_2_factors._preddf
predict.to_csv('/home/mw/project/nn3_tuning_predict.csv')